# Web Analytics (2IID0) Homework Assignment 3
## Exceptional Model Mining vs A/B Testing
#### <i>Abdel K. Bokharouss, Bart van Helvert, Joris Rombouts & Remco Surtel</i>   -   January 2018

In this assignment we will combine Exceptional Model Mining (EMM) and A/B Testing. The dataset has been provided by StudyPortals. The core concept of A/B Testing is that each test subject gets a corresponding variant assigned. After that, we measure the rate of success per variant and the variant with most success is kept, while the other is discarded. However, subgroups that might be served better by the losing version will be disadvantaged. To do better than this, we further mine the data to find coherent subgroups where alternative delivers more success. New visitors to the website that correspond to a specific subgroup, will get either version A or B of the website. Exceptional Model Mining allows us to mine the data further, so that we can discover these subgroups. First, we need to make a decision which attributes will be used as descriptors and which of the attributes will be used as targets.

<font color = "red">Search for "to-do" (Ctrl-F) to find the to-do's that need some evaluation/action</font>

### <font color="green">imports, preparation and configuration</font>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# 1 Data Preprocessing

<b><i>a. Which columns, available in which of the input files, must we designate as our targets in the EMM process?</i></b><br>
The raw data that StudyPortals delivered, contains of three datasets: `clicking_data`, `experiment_details` and `meta_data`. The primary imporance to StudyPortals is analyzing the association between which version of the website were shown (version A or B) and whether or not the user clicked on the button. Therefore the binary attribute `action` of the dataset `clicking_data` is the first target attribute we define. However, we also want to know which version (A or B) was showed to the user. Therefore the attribute `condition` of the dataset `experiment_details` is the second target attribute we define.

<b><i>b.</i></b><br>After the targets are defined, the next step to do is to select which attributes will be used as descriptors. The goal is to gather as much descriptor informaiton as is reasonably possible. First we explore the data, and check how many NAN-values the three datasets contain.

In [2]:
df_click = pd.read_csv('data/clicking_data.csv')
df_click.head()

,action,action_label,action_type,tstamp,user_session
0,clic,revenue,link,1472755490,379881d5-32d7-49f4-bf5b-81fefbc5fcce
1,clic,revenue,link,1472839117,2a0f4218-4f62-479b-845c-109b2720e6e7
2,clic,revenue,link,1472879219,a511b6dc-2dca-455b-b5e2-bf2d224a5505
3,clic,revenue,link,1472890876,9fb616a7-4e13-4307-ac92-0b075d7d376a
4,clic,revenue,link,1472892380,64816772-688d-4460-a591-79aa49bba0d5


In [3]:
df_click.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 5 columns):
action          919 non-null object
action_label    919 non-null object
action_type     919 non-null object
tstamp          919 non-null int64
user_session    919 non-null object
dtypes: int64(1), object(4)
memory usage: 36.0+ KB


In [4]:
df_experiment = pd.read_csv('data/experiment_details_new.csv') # to-do: need to use "_new" dataset?
df_experiment.head()

,user_id,experiment_id,condition,collector_tstamp
0,7794c3b2-4d08-4bb8-bfc9-b50935fed1fc,86-v2-Butonny buttons,1-Control,2016-09-01 12:21:25.716000
1,0baf5074-74bd-4257-ac9f-a07d25f37667,86-v2-Butonny buttons,2-Buttony-Conversion-Buttons,2016-09-01 12:22:01.726000
2,623d19a6-64b4-412a-8143-750995742605,86-v2-Butonny buttons,2-Buttony-Conversion-Buttons,2016-09-01 12:22:31.797000
3,d4b62fc9-dead-441f-936a-db08c2711a1e,86-v2-Butonny buttons,2-Buttony-Conversion-Buttons,2016-09-01 12:22:53.218000
4,8de611c2-0e10-4f63-a016-0f1bd7a683e1,86-v2-Butonny buttons,1-Control,2016-09-01 12:24:15.978000


In [5]:
df_experiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14847 entries, 0 to 14846
Data columns (total 4 columns):
user_id             14764 non-null object
experiment_id       14847 non-null object
condition           14847 non-null object
collector_tstamp    14847 non-null object
dtypes: object(4)
memory usage: 464.0+ KB


In [6]:
#df_meta = pd.read_csv('data/meta_data_modified.csv', encoding = 'ansi')
df_meta = pd.read_table('data/meta_data.tsv', encoding='latin-1')
df_meta.head()

,platform,etl_tstamp,collector_tstamp,dvce_created_tstamp,event,domain_userid,domain_sessionid,user_id,geo_country,geo_region,...,pp_yoffset_max,useragent,browser_language,browser_cookies,browser_colordepth,browser_viewdepth,browser_viewheight,os_name,os_timezone,dvce_type
0,web,06:22.2,06:33.7,06:26.2,page_ping,e8702985-ffee-4ede-a89f-396312539812,0ca47d52-1375-4346-ab2e-b205b64f642e,bc5effb7-a476-414f-a62e-5479022e7553,GR,,...,915.0,Mozilla/5.0 (Linux; Android 4.4.4; GT-I9300 Bu...,el-GR,True,32,360.0,510.0,Android 4.x,Europe/Helsinki,Mobile
1,web,06:49.2,07:00.2,06:52.6,page_ping,e8702985-ffee-4ede-a89f-396312539812,0ca47d52-1375-4346-ab2e-b205b64f642e,bc5effb7-a476-414f-a62e-5479022e7553,GR,,...,1133.0,Mozilla/5.0 (Linux; Android 4.4.4; GT-I9300 Bu...,el-GR,True,32,360.0,615.0,Android 4.x,Europe/Helsinki,Mobile
2,web,07:09.2,07:20.1,07:12.6,page_ping,e8702985-ffee-4ede-a89f-396312539812,0ca47d52-1375-4346-ab2e-b205b64f642e,bc5effb7-a476-414f-a62e-5479022e7553,GR,,...,3216.0,Mozilla/5.0 (Linux; Android 4.4.4; GT-I9300 Bu...,el-GR,True,32,360.0,510.0,Android 4.x,Europe/Helsinki,Mobile
3,web,09:49.3,10:01.0,09:53.4,page_ping,e8702985-ffee-4ede-a89f-396312539812,0ca47d52-1375-4346-ab2e-b205b64f642e,bc5effb7-a476-414f-a62e-5479022e7553,GR,,...,3411.0,Mozilla/5.0 (Linux; Android 4.4.4; GT-I9300 Bu...,el-GR,True,32,360.0,510.0,Android 4.x,Europe/Helsinki,Mobile
4,web,10:40.3,10:51.8,10:44.4,page_ping,e8702985-ffee-4ede-a89f-396312539812,0ca47d52-1375-4346-ab2e-b205b64f642e,bc5effb7-a476-414f-a62e-5479022e7553,GR,,...,0.0,Mozilla/5.0 (Linux; Android 4.4.4; GT-I9300 Bu...,el-GR,True,32,360.0,510.0,Android 4.x,Europe/Helsinki,Mobile


In [7]:
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171949 entries, 0 to 171948
Data columns (total 30 columns):
platform               171949 non-null object
etl_tstamp             171949 non-null object
collector_tstamp       171949 non-null object
dvce_created_tstamp    171949 non-null object
event                  171949 non-null object
domain_userid          171949 non-null object
domain_sessionid       171949 non-null object
user_id                171949 non-null object
geo_country            171690 non-null object
geo_region             171949 non-null object
geo_city               88981 non-null object
geo_region_name        87899 non-null object
geo_timezone           165463 non-null object
page_url               171949 non-null object
page_title             171943 non-null object
page_referrer          155101 non-null object
refr_source            29713 non-null object
pp_xoffset_min         111222 non-null float64
pp_xoffset_max         111222 non-null float64
pp_yoffset_min  

In [8]:
# sum(df_meta.apply(lambda x: sum(x.isnull().values), axis = 1) > 0) # number of rows with NaN values

This whole assignment is about the analysis of the behavior of visitors in terms of an unusual association between which version of the website they were shown (version A or B) and whether or not they clicked on a particular button. Before we actually make an assesment of the best possible strategy for the missing values, we first need to evaluate the volume and completeness of the attributes which identify a specific visitor since the visitors are at the core of this analysis.

In [9]:
df_meta[['domain_userid', 'domain_sessionid', 'user_id']].apply(
    pd.Series.nunique) # unique number of values for each column

domain_userid        8666
domain_sessionid    10310
user_id              9373
dtype: int64

In [10]:
"unique user-sessions in click data: ", len(df_click.user_session.unique())

('unique user-sessions in click data: ', 778)

In [11]:
"unique user-id's in experiment data: ", len(df_experiment.user_id.unique())

("unique user-id's in experiment data: ", 9361)

Each user is identified with a unique user id and each visit is idenitified by a unique user session. Since we are interested in whether an user which visits the website in a certain version, clicks on a particular button or not during a session, we want to limt ourselves to the session id's for which action labels are availabe. We can see that the number of user-id's and session-id's in the three different datasets are not in line with eachother. The goal is to acquire a dataframe in which we have for each user session and id, the experiment details and all the relevant information we can obtain from the meta data file.

### First step: Removing users who have seen two versions 

It is customary in A/B testing and similar research to remove users that have seen both versions of the web page, as is explained in the referenced paper on A/B- and A&B-testing with EMM. This is the first step that we are going to make in the pre-processing part of the assignment. Note that we can obtain the version to which users are exposed from the experiment dataset. If a user (identified with a particular <i>user_id</i> in this problem context) has different entries in the experiment dataset with different values in the <i>condition</i> column (website version indicator), this user should be removed from the dataset(s) which is going to be used in this assignment.

In [12]:
pd.concat(g for _, g in df_click.groupby("user_session") if len(g) > 1).shape # just an oberservation

(235, 5)

From the previous statement we can conclude that there are multiple instances of the same unique <i>user_session</i> identifier in the clicking data. This is already an indicator that having users who have seen the two versions of the website can be deteriorating for the research results. Let's start looking for the users (id's, sessions) in question.

In [13]:
two_versions = df_experiment.copy()
two_versions['count'] = 1 # new column to be used later
two_versions = two_versions.pivot_table(index = 'user_id', columns = 'condition', aggfunc = sum)
two_versions.fillna(0, inplace = True)
two_versions["two_versions_one_userid"] = np.where((((two_versions[('count', '1-Control')] > 0) & (two_versions[('count', '2-Buttony-Conversion-Buttons')] == 0)) |
                                            ((two_versions[('count', '1-Control')] == 0) & (two_versions[('count', '2-Buttony-Conversion-Buttons')] > 0))), 0, 1)
have_seen_two_versions = list(two_versions.loc[two_versions.two_versions_one_userid == 1].index.unique())

We now have a list of users (<i>have_seen_two_versions</i>) and the actual removal of these users is going to be done in the next step.

In [14]:
len(have_seen_two_versions)

233

### Second step: Merging the clicking data with the experiment details

Unfortunately, there are quite a few users who would be deteriorating to the research results. Te clicking dataset is already relatively small, let's hope we do not use too many instances of this dataset.

In [15]:
len(list(set(have_seen_two_versions) & set(df_click.user_session.unique())))

16

As can be seen from the previous statement, removing these users from the clicking data will only result in the loss of clicking/experiment data (click or view) of 16 users (id's, sessions)

Aside from the user_id/sessions in the list <i>have_seen_two_versions</i>, each unique idenitifier in the clicking dataset will be associated with at most one version (<i>condition</i>) as can be found in the experiment datasets. Let's make a dataframe with two relevant columns; one being the unique the identifier and the other column being the condition (version). This will only work for the users who have seen one version. So these users need to be removed first.

In [16]:
users_conditions = df_experiment.copy()
users_conditions = users_conditions[-users_conditions.user_id.isin(have_seen_two_versions)] # drop user who have seen more than one version

In [17]:
users_conditions.drop_duplicates(subset = ['user_id'], keep = 'first', inplace = True)
users_conditions.reset_index(drop = True, inplace = True)

We now have a dataframe <i>user_conditions</i> in which have all the unique user_id's/session which were exposed to this experiment (and to on version). The next step is to merge this dataset with the clicking data. We then will know the condition (website version) of particular user_id's/sessions which resulted in a click/view for each entry in the clicking dataset.

In [18]:
clicking_conditions = pd.merge(left = df_click, right = users_conditions, left_on = 'user_session', right_on = 'user_id')
clicking_conditions.drop(['user_session', 'experiment_id'], axis = 1, inplace = True) # to-do: can also remove action_label, action_type

We can obviously delete some columns, since they are not relevant to the further analysis.

In [19]:
clicking_conditions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 902 entries, 0 to 901
Data columns (total 7 columns):
action              902 non-null object
action_label        902 non-null object
action_type         902 non-null object
tstamp              902 non-null int64
user_id             902 non-null object
condition           902 non-null object
collector_tstamp    902 non-null object
dtypes: int64(1), object(6)
memory usage: 56.4+ KB


### Third step: enriching the clicking data with meta data

The third and final preprocessing step is to enrich the (merged) clicking and experiment data with as much meta-data as is possible. the main objective of this assignment is to identify subpopulations where these targets display an unusual interaction; can we find subgroups where the click rate interacts exceptionally with the web page version (condition). To find these subgroups we need to have suitable descriptors, which can and will be mostly obtained from the meta data. Since things such as device characteristics, location information, language data. This information can be used for A&B Testing (with Exceptional Model Mining) since this data can be queried before exposing a new user (session/id) to one of the versions, based on these characteristics which put the particular user in a certain subgroup if all goes well.

We can merge the subset with these columns with the clicking data by using the user_id and the collector_tstamp. Note that the clicking data did not have a collector_tstamp attribute which makes it hard to be a hundred percent sure whether the particular action is made during a particular session, but the least we can do is make sure that the meta-data info such as the page_url and page_title is confirming to the experiment dataset (things such as geo information will probably be less prone to changes).

In [20]:
df_meta.loc[df_meta.user_id == '379881d5-32d7-49f4-bf5b-81fefbc5fcce'].head(3)

,platform,etl_tstamp,collector_tstamp,dvce_created_tstamp,event,domain_userid,domain_sessionid,user_id,geo_country,geo_region,...,pp_yoffset_max,useragent,browser_language,browser_cookies,browser_colordepth,browser_viewdepth,browser_viewheight,os_name,os_timezone,dvce_type
4441,web,55:48.3,56:05.0,55:30.6,page_view,5346c722-7418-4c41-8ce8-f17ac198ff05,ce5519f6-4952-4722-8a04-b186f856f640,379881d5-32d7-49f4-bf5b-81fefbc5fcce,CY,,...,NaN,Mozilla/5.0 (Linux; Android 5.1.1; SM-G531F Bu...,el,True,32,360.0,560.0,Android 5.x,Europe/Helsinki,Mobile
4444,web,56:08.3,56:24.6,55:50.6,page_ping,5346c722-7418-4c41-8ce8-f17ac198ff05,ce5519f6-4952-4722-8a04-b186f856f640,379881d5-32d7-49f4-bf5b-81fefbc5fcce,CY,,...,3285.0,Mozilla/5.0 (Linux; Android 5.1.1; SM-G531F Bu...,el,True,32,360.0,616.0,Android 5.x,Europe/Helsinki,Mobile
5107,web,23:42.0,23:59.0,23:24.8,page_ping,5346c722-7418-4c41-8ce8-f17ac198ff05,fcdcd0c3-7c31-4a4d-aee7-0631c73dec92,379881d5-32d7-49f4-bf5b-81fefbc5fcce,CY,,...,1804.0,Mozilla/5.0 (Linux; Android 5.1.1; SM-G531F Bu...,el,True,32,360.0,616.0,Android 5.x,Europe/Helsinki,Mobile


In [21]:
clicking_conditions.loc[clicking_conditions.user_id == '379881d5-32d7-49f4-bf5b-81fefbc5fcce']

,action,action_label,action_type,tstamp,user_id,condition,collector_tstamp
0,clic,revenue,link,1472755490,379881d5-32d7-49f4-bf5b-81fefbc5fcce,1-Control,2016-09-01 18:43:14.257000


One can, however, see that the timestamp formats of the two data set are not conforming. In addition, even if when one could go as far formatting the <i>collector_tstamp</i> to the same format as the time stamps used in the meta dataset, one would still find different / conflicting timestamps. This was later on confirmed and explained by Tara from StudyPortals: <i>"Timestamp in the experiment file indicates the timestamp that the users starts the session while seeing the variation A or B. I recommend you to use the meta-data timestamp for your further analysis. "</i>. We are thus just going to merge on the user-id's.

Since we cannot use the timestamp value to link this dataset with the previous one. We are only after the user_id's. The following statement is an important step before the actual merger with the already enriched clicking data.

In [22]:
users_meta = df_meta.groupby('user_id').first().reset_index()

In [23]:
# users_meta.info()

This statement gives us the same dataframe with the unique user_id's takes the first non-NaN value in for each column of a particular user_id. Each user_id will thus have a non-NaN value in the column if it has a non-NaN value for this column in one of its occurences in <i>df_meta</i>

This dataset can now be merged with the dataset from step 2.

In [24]:
click_condition_meta = pd.merge(clicking_conditions, users_meta, on = 'user_id')

In [25]:
click_condition_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 902 entries, 0 to 901
Data columns (total 36 columns):
action                 902 non-null object
action_label           902 non-null object
action_type            902 non-null object
tstamp                 902 non-null int64
user_id                902 non-null object
condition              902 non-null object
collector_tstamp_x     902 non-null object
platform               902 non-null object
etl_tstamp             902 non-null object
collector_tstamp_y     902 non-null object
dvce_created_tstamp    902 non-null object
event                  902 non-null object
domain_userid          902 non-null object
domain_sessionid       902 non-null object
geo_country            900 non-null object
geo_region             902 non-null object
geo_city               552 non-null object
geo_region_name        534 non-null object
geo_timezone           866 non-null object
page_url               902 non-null object
page_title             902 non-null 

Let's first make an assesmment of columns in the meta dataset which are definitely not usable

In [26]:
print(click_condition_meta.platform.unique()) # one unique value for the entire dataframe
click_condition_meta.drop(['platform'], axis = 1, inplace = True)

['web']


An attribute fit to be a candidate descriptor should be available before or while loading the page since the main goal of A&B Testing is to have a dynamic web page / page elements. Therefore, some columns can already be excluded from the list of potential descriptors in the next task. Take for example the scrolling characteristics. One cannot expect these continiuous values to be of any use as descriptors. In addition, the timestamps and session info are also no longer relevant

In [27]:
not_fit = ['pp_xoffset_min', 'pp_yoffset_min', 'pp_xoffset_max', 'pp_yoffset_max', 'tstamp',
          'collector_tstamp_x', 'etl_tstamp', 'collector_tstamp_y', 'dvce_created_tstamp', 'event', 'domain_userid',
          'domain_sessionid']
consistent_with_action = ['action_label', 'action_type']
click_condition_meta.drop(not_fit + consistent_with_action, axis = 1, inplace = True)

The browser information can be used to load dynamic pages. One can for example use JavaScript for this purpose (if you want to handle this, for example, on the front-end (but on the server side would be better)). The colordepth can, for example be obtained by statements in the language, and this value can then be used in logical statements (<i>object.colorDepth</i>) to load a certain webpage version/elements.

In [28]:
click_condition_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 902 entries, 0 to 901
Data columns (total 21 columns):
action                902 non-null object
user_id               902 non-null object
condition             902 non-null object
geo_country           900 non-null object
geo_region            902 non-null object
geo_city              552 non-null object
geo_region_name       534 non-null object
geo_timezone          866 non-null object
page_url              902 non-null object
page_title            902 non-null object
page_referrer         842 non-null object
refr_source           654 non-null object
useragent             902 non-null object
browser_language      902 non-null object
browser_cookies       902 non-null bool
browser_colordepth    902 non-null int64
browser_viewdepth     902 non-null float64
browser_viewheight    902 non-null float64
os_name               902 non-null object
os_timezone           902 non-null object
dvce_type             902 non-null object
dtypes: bool(1

Let's check the remaining attributes and there values in the dataset to exclude some more potential descriptors.

In [29]:
show_max = 20
columns_meta = list(set(click_condition_meta.columns) - set(not_fit) - set(clicking_conditions.columns))
for c in columns_meta:
    print(c, '\n', "unique number of values:", len(click_condition_meta[c].unique()), '\n',
          list(click_condition_meta[c].unique())[:show_max], '\n')

useragent 
 unique number of values: 404 
 ['Mozilla/5.0 (Linux; Android 5.1.1; SM-G531F Build/LMY48B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.98 Mobile Safari/537.36', 'Mozilla/5.0 (iPhone; CPU iPhone OS 9_3_4 like Mac OS X) AppleWebKit/601.1 (KHTML, like Gecko) CriOS/52.0.2743.84 Mobile/13G35 Safari/601.1.46', 'Opera/9.80 (Android; Opera Mini/10.0.1884/37.8861; U; en) Presto/2.12.423 Version/12.16', 'Mozilla/5.0 (iPhone; CPU iPhone OS 9_3_4 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Version/9.0 Mobile/13G35 Safari/601.1', 'Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-J700H Build/MMB29K) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/4.0 Chrome/44.0.2403.133 Mobile Safari/537.36', 'Mozilla/5.0 (iPhone; CPU iPhone OS 9_3_5 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Version/9.0 Mobile/13G36 Safari/601.1', 'Mozilla/5.0 (Linux; Android 5.0; Lenovo A1000 Build/S100) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.93 Mobile Safari/53

1. <b>browser_language</b> seems to be fit to use as a potential descriptor. There attribute needs, however, some flattening. The most reasonable thing to do is to make the follwing categories: English, Latin-Non-English, Cytllic (i.e. Russian etc), Arabic and Asian
2. <b>os_timezone</b> seems to be fit to use as a potential descriptor
3. <b>os_name</b> seems to be fit to use as a potential descriptor, but needs some feature engineering (i.e. flaten sub-OSs into one OS (for example, different Android Versions to just "Android")). This will increase the attribute's (descriptor) usability later on + has a lot of NaN values
4. <b>brower_color_depth</b> can be used, but it would be a very far fetch to assume that the color depth will be a decent descriptor of any subgroup. The choice is, therefore, made to discard it as potential descriptor.
5. <b>refr_source</b> seems to be fit to use as a potential descriptor. To-do: Check whether a nan-value for this attribute implies that the reference was internal (i.e. clicking though website) (use the <i>page_referrer</i> attribute)
6. <b>browser_cookies</b> has one unique value (True) and is, therefore, not fit as descriptor
7. <b>geo_city</b> has a lot of unique values. One is better of using the higher level location attributes. The choice is, therefore, to discard as potential descriptor.
8. <b>dvce_type</b> seems to be fit to use as a potential descriptor
9. <b>page_referrer</b> is in itself not particular fit to be used as decriptor, but can be used to complete/improve the usability of the refr_source attribute.
10. <b>geo_region/geo_region_name</b> have a lot of unique values. One is better of using the higher level location attributes
11. <b>useragent</b> is in itself not very useful as a descriptor, but can be used to complete the os_name info, if necessary.
12. <b>browser_viewdepth/browser_viewheight</b> could be used, but these attributes need to be considered in pairs and need some binning.
13. <b>geo_timezone</b> seems also usable, but we already have the os_timezone, which is of the same format and holds the same information. We can, however, check whether we can use this attribute to fill in NaN values for the os_timezone.

### browser_language

In [30]:
languages = list(click_condition_meta.browser_language.unique())
languages =  [x.lower() for x in languages] # lower case
english = [s for s in languages if 'en' in s]
languages = list(set(languages) - set(english)) # remove english from the set
cyrillic = ['uk-ua', 'ru', 'bg', 'uk', 'hr-hr', 'ru-ru']
asian = ['zh-cn', 'ja', 'zh-tw', 'ja-jp', 'vi-vn', 'vi', 'ko', 'th-th', 'id-id', 'id']
greek = ['el-gr', 'el']
herbrew = ['he-il', 'he']
latin_non_en = ['it', 'sk-sk', 'ro', 'pt-br', 'sv-se', 'ro-ro', 'es', 'tr', 'es-es', 'pl-pl', 'es-xl', 'nb', 'fr-fr',
                'it-it', 'es-419', 'da-dk', 'fr', 'de', 'nl-nl', 'tr-tr', 'nl', 'de-de', 'ar']

In [31]:
click_condition_meta.browser_language = click_condition_meta.browser_language.str.lower()

In [32]:
click_condition_meta['browser_language'] = np.where(click_condition_meta['browser_language'].isin(english), 'english', click_condition_meta.browser_language)
click_condition_meta['browser_language'] = np.where(click_condition_meta['browser_language'].isin(cyrillic), 'cyrillic', click_condition_meta.browser_language)
click_condition_meta['browser_language'] = np.where(click_condition_meta['browser_language'].isin(asian), 'asian', click_condition_meta.browser_language)
click_condition_meta['browser_language'] = np.where(click_condition_meta['browser_language'].isin(greek), 'greek', click_condition_meta.browser_language)
click_condition_meta['browser_language'] = np.where(click_condition_meta['browser_language'].isin(herbrew), 'herbrew', click_condition_meta.browser_language)
click_condition_meta['browser_language'] = np.where(click_condition_meta['browser_language'].isin(latin_non_en), 'latin_lan', click_condition_meta.browser_language)

In [33]:
click_condition_meta.browser_language.unique()

array(['greek', 'english', 'latin_lan', 'asian', 'cyrillic', 'herbrew'], dtype=object)

### os_name

In [34]:
os_names = list(click_condition_meta.os_name.unique())
os_names =  [x.lower() for x in os_names] # lower case
ios = [s for s in os_names if 'iphone' in s]
android = [s for s in os_names if 'android' in s]
windows = [s for s in os_names if 'windows' in s]
other = [s for s in os_names if 'unknown' in s]

In [35]:
click_condition_meta.os_name = click_condition_meta.os_name.str.lower()

In [36]:
click_condition_meta['os_name'] = np.where(click_condition_meta['os_name'].isin(ios), 'iOS', click_condition_meta.os_name)
click_condition_meta['os_name'] = np.where(click_condition_meta['os_name'].isin(android), 'Android', click_condition_meta.os_name)
click_condition_meta['os_name'] = np.where(click_condition_meta['os_name'].isin(windows), 'Windows', click_condition_meta.os_name)
click_condition_meta['os_name'] = np.where(click_condition_meta['os_name'].isin(other), 'Other', click_condition_meta.os_name)

### geo_country

In [37]:
# can be done more elegantly/thoroughly, but enough advanced pre-processing steps have been illustrated
click_condition_meta['geo_country'] = np.where(click_condition_meta['os_timezone'] == 'Europe/London' , 'GB', click_condition_meta.geo_country)
click_condition_meta['geo_country'] = np.where(click_condition_meta['os_timezone'] == 'Europe/Helsinki' , 'FI', click_condition_meta.geo_country)
click_condition_meta['geo_country'] = np.where(click_condition_meta['os_timezone'] == 'Europe/Berlin' , 'DE', click_condition_meta.geo_country)
click_condition_meta['geo_country'] = np.where(click_condition_meta['os_timezone'] == 'Asia/Kolkata' , 'IN', click_condition_meta.geo_country)
click_condition_meta['geo_country'] = np.where(click_condition_meta['os_timezone'] == 'Asia/Karachi' , 'PK', click_condition_meta.geo_country)
click_condition_meta['geo_country'] = np.where(click_condition_meta['os_timezone'] == 'America/Santo_Domingo' , 'DO', click_condition_meta.geo_country)
click_condition_meta['geo_country'] = np.where(click_condition_meta['os_timezone'] == 'Asia/Beirut' , 'LB', click_condition_meta.geo_country)
click_condition_meta['geo_country'] = np.where(click_condition_meta['os_timezone'] == 'Asia/Dubai' , 'AE', click_condition_meta.geo_country)

### not fit

In [38]:
click_condition_meta.drop(['geo_timezone', 'browser_cookies', 'geo_region', 'browser_colordepth', 'geo_city',
                           'geo_region_name', 'useragent', 'browser_viewdepth', 'browser_viewheight'], axis = 1, inplace = True)

### os_timezone

In [39]:
click_condition_meta.os_timezone.unique()

array(['Europe/Helsinki', 'Australia/Sydney', 'Europe/London',
       'Asia/Dhaka', 'Africa/Lagos', 'Africa/Cairo', 'Europe/Berlin',
       'Asia/Shanghai', 'America/Bogota', 'Asia/Baghdad',
       'America/Sao_Paulo', 'Asia/Karachi', 'UTC', 'America/New_York',
       'Europe/Minsk', 'Asia/Tokyo', 'Asia/Beirut', 'America/Chicago',
       'Asia/Kolkata', 'America/Denver', 'America/Goose_Bay',
       'America/Los_Angeles', 'Asia/Jakarta', 'Africa/Johannesburg',
       'Asia/Rangoon', 'America/Asuncion', 'Asia/Baku', 'Asia/Kathmandu',
       'Europe/Moscow', 'Asia/Jerusalem', 'America/Mexico_City',
       'America/Santo_Domingo', 'Asia/Dubai', 'Australia/Brisbane',
       'America/Guatemala', 'America/Phoenix', 'Asia/Krasnoyarsk',
       'America/Montevideo', 'Africa/Windhoek', 'Australia/Adelaide'], dtype=object)

In [40]:
timezones = list(click_condition_meta.os_timezone.unique())
timezones = [x.lower() for x in timezones]
europe = [s for s in timezones if 'europe' in s] + ['utc']
america = [s for s in timezones if 'america' in s]
asia = [s for s in timezones if 'asia' in s]
australia = [s for s in timezones if 'australia' in s]
africa = [s for s in timezones if 'africa' in s]

In [41]:
click_condition_meta.os_timezone = click_condition_meta.os_timezone.str.lower()

In [42]:
click_condition_meta['os_timezone'] = np.where(click_condition_meta['os_timezone'].isin(europe), 'Europe', click_condition_meta.os_timezone)
click_condition_meta['os_timezone'] = np.where(click_condition_meta['os_timezone'].isin(america), 'America', click_condition_meta.os_timezone)
click_condition_meta['os_timezone'] = np.where(click_condition_meta['os_timezone'].isin(asia), 'Asia', click_condition_meta.os_timezone)
click_condition_meta['os_timezone'] = np.where(click_condition_meta['os_timezone'].isin(australia), 'Australia', click_condition_meta.os_timezone)
click_condition_meta['os_timezone'] = np.where(click_condition_meta['os_timezone'].isin(africa), 'Africa', click_condition_meta.os_timezone)

In [43]:
click_condition_meta.os_timezone.unique()

array(['Europe', 'Australia', 'Asia', 'Africa', 'America'], dtype=object)

### devc_type

In [44]:
click_condition_meta.loc[click_condition_meta.dvce_type == "Unknown"].head(2)

,action,user_id,condition,geo_country,page_url,page_title,page_referrer,refr_source,browser_language,os_name,os_timezone,dvce_type
403,clic,1f336e8c-d658-4656-bd87-aae8995e2725,2-Buttony-Conversion-Buttons,GM,http://www.mastersportal.eu/countries/82/unite...,Masters in United States - MastersPortal.eu,http://www.mastersportal.eu/countries/82/unite...,NaN,english,Other,Europe,Unknown
704,view,ed1b6397-0c3d-4efe-9e2f-f448b60a7384,2-Buttony-Conversion-Buttons,US,http://www.mastersportal.eu/universities/1145/...,Polimoda Institute of Fashion Design and Marke...,NaN,NaN,english,Other,America,Unknown


In [45]:
click_condition_meta['dvce_type'] = np.where(click_condition_meta.user_id == "1f336e8c-d658-4656-bd87-aae8995e2725", 'Mobile', click_condition_meta.dvce_type)

Apparently, one of the users is a GoogleBot (webcrawling bot of google used for for example google search indexing). This user can be removed, since you do not want to take this user into account in the research.

In [46]:
click_condition_meta = click_condition_meta[click_condition_meta.dvce_type != 'Unknown'] # google bot

In [47]:
click_condition_meta.dvce_type.unique()

array(['Mobile', 'Tablet', 'Computer'], dtype=object)

### refr_source

In [48]:
click_condition_meta[pd.isnull(click_condition_meta['refr_source'])].shape[0]

245

We are going to fill in this attribute by using the <i>page_referrer</i> attribute to reduce the number of null values of this attribute. If the <i>page_referrer</i> is, for example, a search engine, we can fille in this search engine as <i>refr_source</i>. If the <i>page_referrer</i> is a website from the same domain, the reference was internally.
The reason for a NaN value for the <i>page_referrer</i> is diffult to specify. Just to list some reasons:
* The user just entered the url belonging to the page themself / used a browser-maintained bookmark
* The user made a transition from a secure HTTPS connection/page/website to an 'insecure' one (just HTTP)
* The user has software (e.g. anti-virus or firewall) which stripped the <i>page_referrer</i> info from the HTTP request
* ... There are many more reasons, but this course is not about web-technology or computer networks ;)

In [49]:
references = list(click_condition_meta.page_referrer.unique())
references = [str(s) for s in references]
google = [s for s in references if 'google' in s]
portal = [s for s in references if 'portal' in s]
facebook = [s for s in references if 'facebook' in s]
baidu = [s for s in references if 'baidu' in s]
yahoo = [s for s in references if 'yahoo' in s]
everyclick = [s for s in references if 'everyclick' in s]
duckduckgo = [s for s in references if 'duckduckgo' in s]
click_condition_meta.page_referrer = click_condition_meta.page_referrer.str.lower()

In [50]:
click_condition_meta.refr_source.unique()

array([nan, 'Google', 'Yahoo', 'Yandex', 'Bing', 'DuckDuckGo', 'Facebook',
       'Vkontakte', 'Everyclick'], dtype=object)

In [51]:
click_condition_meta['refr_source'] = np.where(((click_condition_meta['page_referrer'].isin(google))
                                                & pd.isnull(click_condition_meta['refr_source'])),
                                               'Google', click_condition_meta.refr_source)
click_condition_meta['refr_source'] = np.where(((click_condition_meta['page_referrer'].isin(portal))
                                                & pd.isnull(click_condition_meta['refr_source'])),
                                               'StudyPortal', click_condition_meta.refr_source)
click_condition_meta['refr_source'] = np.where(((click_condition_meta['page_referrer'].isin(facebook))
                                                & pd.isnull(click_condition_meta['refr_source'])),
                                               'Facebook', click_condition_meta.refr_source)
click_condition_meta['refr_source'] = np.where(((click_condition_meta['page_referrer'].isin(baidu))
                                                & pd.isnull(click_condition_meta['refr_source'])),
                                               'Baidu', click_condition_meta.refr_source)
click_condition_meta['refr_source'] = np.where(((click_condition_meta['page_referrer'].isin(yahoo))
                                                & pd.isnull(click_condition_meta['refr_source'])),
                                               'Yahoo', click_condition_meta.refr_source)
click_condition_meta['refr_source'] = np.where(((click_condition_meta['page_referrer'].isin(everyclick))
                                                & pd.isnull(click_condition_meta['refr_source'])),
                                               'Everyclick', click_condition_meta.refr_source)
click_condition_meta['refr_source'] = np.where(((click_condition_meta['page_referrer'].isin(duckduckgo))
                                                & pd.isnull(click_condition_meta['refr_source'])),
                                               'DuckDuckGo', click_condition_meta.refr_source)
click_condition_meta.drop(['page_referrer', 'page_title', 'page_url'], axis = 1, inplace = True) # not needed anymore

In [52]:
print("filled in", 245 - click_condition_meta[pd.isnull(click_condition_meta['refr_source'])].shape[0], "NaN values")

filled in 172 NaN values


So this data preprocessing step for the reference source was quite useful. The remaining NaN values fall under the various cases which have been explained a few cells ago. Let's classifiy these cases as just "refr_source not available".

In [53]:
click_condition_meta['refr_source'] = np.where((pd.isnull(click_condition_meta['refr_source'])),
                                               'refr_source not available', click_condition_meta.refr_source)

### final pre-processing steps

In [54]:
(pd.DataFrame(click_condition_meta.groupby('geo_country').count())).sort_values('action', ascending = False).head(4)

,action,user_id,condition,refr_source,browser_language,os_name,os_timezone,dvce_type
geo_country,,,,,,,,
DE,172,172,172,172,172,172,172,172
GB,100,100,100,100,100,100,100,100
FI,87,87,87,87,87,87,87,87
IN,67,67,67,67,67,67,67,67


There are still two NaN values in the geo_country column. One can observe that these vistors are from Africa. One can safely assume that these users are from Nigeria since the website is relatively popular among Nigerians (conclusion based on data: 4th most frequent country as can be seen in the cell above) and the frequency of other countries in Africa are rather low.

In [55]:
click_condition_meta['geo_country'] = np.where((pd.isnull(click_condition_meta['geo_country'])),
                                               'NG', click_condition_meta.geo_country)

In [56]:
click_condition_meta.reset_index(inplace = True, drop = True)
click_condition_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899 entries, 0 to 898
Data columns (total 9 columns):
action              899 non-null object
user_id             899 non-null object
condition           899 non-null object
geo_country         899 non-null object
refr_source         899 non-null object
browser_language    899 non-null object
os_name             899 non-null object
os_timezone         899 non-null object
dvce_type           899 non-null object
dtypes: object(9)
memory usage: 63.3+ KB


In [57]:
click_condition_meta.head(3)

,action,user_id,condition,geo_country,refr_source,browser_language,os_name,os_timezone,dvce_type
0,clic,379881d5-32d7-49f4-bf5b-81fefbc5fcce,1-Control,FI,Google,greek,Android,Europe,Mobile
1,clic,2a0f4218-4f62-479b-845c-109b2720e6e7,2-Buttony-Conversion-Buttons,AU,Google,english,iOS,Australia,Mobile
2,clic,a511b6dc-2dca-455b-b5e2-bf2d224a5505,2-Buttony-Conversion-Buttons,GB,Google,english,Android,Europe,Mobile


So we now have a fuly pre-processed dataframe in which we have the target attributes <i>action</i> and <i>condition</i> and descriptors <i>geo_country, refr_source, browser_language, os_name, os_timezone</i> and <i>dvce_type</i> which are all processed/flattened where considered necessary.

In [58]:
click_condition_meta.to_csv('./data/action_condition_meta.csv', index = False) # for usage in other notebooks and scripts

### 2 A&B Testing

### 3 Beyond A&B Testing